In [1]:
import pandas as pd
import pylab as pl
import json
%pylab inline
pl.rcParams["font.size"] = 20

//anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Populating the interactive namespace from numpy and matplotlib


# Categorical distance metrics

### reading in Women Services database from NYC open data

In [2]:
womenserv = pd.read_csv("https://data.cityofnewyork.us/api/views/pqg4-dm6b/rows.csv?accessType=DOWNLOAD")
#NYC_Women_s_Resource_Network_Database.csv")
womenserv.head()

,OrganizationName,Brooklyn,Bronx,Manhattan,Queens,Staten Island,Fax,Phone,URL,noURL,...,Location 2,Borough,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,YWCA of the City of New York,Y,N,Y,N,Y,2.122236e+09,2.127554e+09,http://www.ywcanyc.org,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RightRides for Women's Safety,Y,Y,Y,Y,N,NaN,7.185221e+09,http://www.rightrides.org,N,...,NaN,BROOKLYN,40.693244,-73.990660,2.0,33.0,9.0,3002094.0,3.002500e+09,Brooklyn Heights-Cobble Hill ...
2,Charles B. Wang Community Health Center,N,N,Y,N,N,NaN,2.129660e+09,http://www.cbwchc.org/hcs/wh/wh.html,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,American-Italian Cancer Foundation's Mobile Ma...,Y,Y,Y,Y,Y,NaN,2.126289e+09,http://www.americanitaliancancer.org,N,...,NaN,MANHATTAN,40.770141,-73.963670,8.0,4.0,128.0,1083890.0,1.014058e+09,Upper East Side-Carnegie Hill ...
4,QHC-Diabetes Center,N,N,N,Y,N,7.188836e+09,7.188833e+09,http://nyc.gov,N,...,NaN,QUEENS,40.717250,-73.803433,8.0,24.0,1267.0,4442431.0,4.068580e+09,Pomonok-Flushing Heights-Hillcrest ...


### encoding variable as a binary
(reference on categorical encoding in python: http://pbpython.com/categorical-encoding.html)

In [3]:
Y21N20 = {'Y':1, 'N':0}
    
for br in 'Brooklyn', 'Bronx', 'Manhattan', 'Queens', 'Staten Island':
    #womenserv[br][womenserv[br] == 'Y'] = 1
    #womenserv[br][womenserv[br] == 'N'] = 0
    womenserv[br] = womenserv[br].map(Y21N20)
womenserv.head()

,OrganizationName,Brooklyn,Bronx,Manhattan,Queens,Staten Island,Fax,Phone,URL,noURL,...,Location 2,Borough,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,YWCA of the City of New York,1,0,1,0,1,2.122236e+09,2.127554e+09,http://www.ywcanyc.org,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RightRides for Women's Safety,1,1,1,1,0,NaN,7.185221e+09,http://www.rightrides.org,N,...,NaN,BROOKLYN,40.693244,-73.990660,2.0,33.0,9.0,3002094.0,3.002500e+09,Brooklyn Heights-Cobble Hill ...
2,Charles B. Wang Community Health Center,0,0,1,0,0,NaN,2.129660e+09,http://www.cbwchc.org/hcs/wh/wh.html,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,American-Italian Cancer Foundation's Mobile Ma...,1,1,1,1,1,NaN,2.126289e+09,http://www.americanitaliancancer.org,N,...,NaN,MANHATTAN,40.770141,-73.963670,8.0,4.0,128.0,1083890.0,1.014058e+09,Upper East Side-Carnegie Hill ...
4,QHC-Diabetes Center,0,0,0,1,0,7.188836e+09,7.188833e+09,http://nyc.gov,N,...,NaN,QUEENS,40.717250,-73.803433,8.0,24.0,1267.0,4442431.0,4.068580e+09,Pomonok-Flushing Heights-Hillcrest ...


## define similarity

In [4]:
def SimpleSimilarity(x,y):
    '''Calculates Simple similarity for categorical binary variables
    Arguments: 
    x: series (array) of categorical observation for an feature
    y: series (array) of categorical observation for an feature
    Returns:
    Simple similarity (float)
    '''
    
    coocc = x + y
    m11 = np.sum(coocc==0) # in neither
    m00 = np.sum(coocc==2) # in both
    p = len(coocc)
    return (m11 + m00) * 1.0 / p

In [5]:
def JaccardSimilarity(x,y):
    '''Calculates Jaccard similarity for categorical binary variables
    Arguments: 
    x: series (array) of categorical observation for an feature
    y: series (array) of categorical observation for an feature
    Returns:
    Jaccard similarity (float)
    '''
    
    coocc = x + y
    a = (coocc==2).sum() # in both
    bc = (coocc==1).sum() #in either but not both
    return (a) * 1.0 / bc

In [9]:
print ("How similar is Brooklyn to the Bronx?")
    
print ("Simple Similarity: ", SimpleSimilarity(womenserv.Brooklyn.values, womenserv.Bronx.values), "\n",
      "Jaccard Similarity: ", JaccardSimilarity(womenserv.Brooklyn.values, womenserv.Bronx.values),
      "\n\n")

print ("How similar is Manhattan to the Bronx?")
    
print ("Simple Similarity: ", SimpleSimilarity(womenserv.Manhattan.values, womenserv.Bronx.values), "\n",
      "Jaccard Similarity: ", JaccardSimilarity(womenserv.Manhattan.values, womenserv.Bronx.values),
      "\n\n")

print ("How similar is Manhattan to the Brooklyn?")
    
print ("Simple Similarity: ", SimpleSimilarity(womenserv.Manhattan.values, womenserv.Brooklyn.values), "\n",
      "Jaccard Similarity: ", JaccardSimilarity(womenserv.Manhattan.values, womenserv.Brooklyn.values),
      "\n\n")

print ("How similar is Manhattan to the SI?")
    
print ("Simple Similarity: ", SimpleSimilarity(womenserv.Manhattan.values, womenserv["Staten Island"].values), "\n",
      "Jaccard Similarity: ", JaccardSimilarity(womenserv.Manhattan.values, womenserv["Staten Island"].values),
      "\n\n")


How similar is Brooklyn to the Bronx?
Simple Similarity:  0.770706190061 
 Jaccard Similarity:  0.733840304183 


How similar is Manhattan to the Bronx?
Simple Similarity:  0.502179598954 
 Jaccard Similarity:  0.353765323993 


How similar is Manhattan to the Brooklyn?
Simple Similarity:  0.426329555362 
 Jaccard Similarity:  0.313069908815 


How similar is Manhattan to the SI?
Simple Similarity:  0.531822144725 
 Jaccard Similarity:  0.329608938547 


